In [13]:
import pandas as pd
import numpy as np
import scipy.stats
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder
# get rid of warnings
import warnings
warnings.filterwarnings("ignore")
# get more than one output per Jupyter cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# for functions we implement later
from utils import best_fit_distribution
from utils import plot_result




ImportError: cannot import name 'best_fit_distribution' from 'utils' (C:\Users\shayden\AppData\Local\Continuum\anaconda3\lib\site-packages\utils\__init__.py)

In [14]:

#import file into DF
Employee_data_DF = pd.read_csv('https://raw.githubusercontent.com/skhayden/MSDS-7349-Sec403-/master/Class_Project/Employee_data.csv',low_memory=False)
#Remove unneeded columns
Employee_data_DF.drop(['Job.1','Unnamed: 0'], axis = 1, inplace = True) 
Employee_data_DF.head()

,First Name,Last Name,Date of Birth,Job,Address,Salary,SSN,Employee_Id,Office_Location,Building_number,Phone_Number,Email
0,Matthew,Johnson,4/11/1993,Research scientist (physical sciences),USS Walker\nFPO AA 91930,64822,691-17-3378,P37429,West Andrew,961,3298685475,Matthewgmail.com
1,Elizabeth,Frazier,6/17/1989,"Therapist, speech and language","8283 Kimberly Streets Apt. 001\nEast Edward, D...",37351,675-40-3501,P90953,South Tammymouth,590,524-601-7567x7090,EFrazier93erez.com
2,Lori,Vargas,8/24/1959,"Designer, furniture","235 Devin Ridges\nCraneshire, OR 16065",74379,821-81-4954,P38017,West Arthur,198,316-095-9037x69942,Loriegregory@smith.com
3,Sarah,Espinoza,12/1/1983,Water engineer,"78762 Eileen Camp\nNelsonhaven, ID 36069",81275,124-10-4970,P91959,Greerborough,10033,+1-209-595-5257x826,SEspinozad-perry.com
4,Melissa,Hunter,9/24/1949,Agricultural engineer,"3346 Anderson Lakes\nShawville, IL 85803",35409,789-99-6633,P36920,Davisburgh,633,44421409,Melissazkimberly@wiley.com


In [18]:
# defines two types of columns; categorical and continuous 
#https://dev.to/r0f1/a-simple-way-to-anonymize-data-with-python-and-pandas-79g
categorical = []
Perterbation_col = []

for c in list(Employee_data_DF):
    col = Employee_data_DF[c]
    nunique = col.nunique()
    if nunique < 20:
        categorical.append(c)
    else:
        Perterbation_col.append(c)


In [15]:
Perterbation_col = []
Perterbation_col.append(Employee_data_DF['Salary'])  

In [20]:
best_distributions = []

for c in Perterbation_col:
    data = Employee_data_DF[c]
    best_fit_name, best_fit_params = best_fit_distribution(data, 50)
    best_distributions.append((best_fit_name, best_fit_params))

NameError: name 'best_fit_distribution' is not defined

In [ ]:
for c in continuous:
    data = df[c]
    best_fit_name, best_fit_params = best_fit_distribution(data, 50)
    best_distributions.append((best_fit_name, best_fit_params))

In [9]:
categorical

[0         64822
 1         37351
 2         74379
 3         81275
 4         35409
 5         77782
 6         78386
 7         43175
 8         44061
 9         55165
 10        65541
 11        75449
 12        35110
 13        55948
 14        62514
 15        46443
 16        87649
 17        84968
 18        66616
 19        36573
 20        74515
 21        87347
 22        66846
 23        51498
 24        72739
 25        84655
 26        44567
 27        53361
 28        46671
 29        84668
           ...  
 20561    398056
 20562    360626
 20563    488057
 20564    363760
 20565    430395
 20566    378292
 20567    235486
 20568    232246
 20569    238195
 20570    446584
 20571    400089
 20572    294996
 20573    320319
 20574    362999
 20575    350850
 20576    316129
 20577    269806
 20578    245687
 20579    212221
 20580    481373
 20581    282530
 20582    203895
 20583    408639
 20584    496705
 20585    389966
 20586    324819
 20587    208342
 20588    4791